<a href="https://colab.research.google.com/github/EricZhou0815/Pytorch-Study/blob/master/nn_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = True # set it to true the first time you build your data, then change to false

# class to normalize the image data
class DogsVSCats():
    IMG_SIZE = 50
    # local colab dir (google drive), need to upload dataset to google drive
    URL="/content/drive/My Drive/Dataset/PetImages/"
    CATS = URL + "Cat"
    DOGS = URL + "Dog"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []
    cat_count = 0
    dog_count = 0

    def make_training_data(self):
      for label in self.LABELS:
        print(label)
        for f in tqdm(os.listdir(label)):
          try:
            path = os.path.join(label,f)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img,(self.IMG_SIZE,self.IMG_SIZE))
            self.training_data.append([np.array(img),np.eye(2)[self.LABELS[label]]])

            if label == self.CATS:
              self.cat_count += 1
            elif label == self.DOGS:
              self.dog_count += 1
          except Exception as e:
            print (e)
      
      np.random.shuffle(self.training_data)
      np.save("training_data.npy",self.training_data)
      print("Cats:",self.cat_count)
      print("Dogs:",self.dog_count)

if REBUILD_DATA:
  dogs_vs_cats = DogsVSCats()
  dogs_vs_cats.make_training_data()

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear ,512)
        self.fc2 = nn.Linear(512,2)

    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))

        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear=x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x,dim = 1)

net=Net()

In [0]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[0] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

In [0]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

In [0]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
  for i in dqdm(range(0,len(train_X),BATCH_SIZE)):
    #print(i,i+BACH_SIZE)
    batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
    batch_y = train_y[i:i+BATCH_SIZE]

    net.zero_grad()
    outputs=net(batch_X)
    loss = loss_function(output,batch_y)
    loss.backward()
    optimizer.step()

print(loss)

In [0]:
correct = 0
totla = 0
with torch.no_grad():
  for i in tqdm(range(len(test_X))):
    real_class = torch.argmax(test_y[i])
    net_out=net(test_X[i].view(-1,1,50,50))[0]
    predicted_class = torch.argmax(net_out)
    if predicted_class == real_class:
      correct += 1
    totla += 1
print("Accuracy:",cound(correct/total,3))
